In [190]:
!python train.py

arena_data	       movielens.py	     tagstext.txt
arena_util.py	       myals.py		     tags.vocab
bm25K2.pkl	       ndcg.txt		     tmp1000.pkl
bm25K6.pkl	       omg2.json	     tmp1000_real.pkl
coo_real.txt	       omg2_real.json	     tmp1.pkl
coo_train_only.txt     only_train.json	     tmp2000.pkl
coo_train_val.txt      __pycache__	     tmp2000_real.pkl
coo.txt		       res		     tmp2.pkl
evaluate.py	       run.py		     train.py
genre_most_popular.py  similar-playlist.txt  utils.py
mask.pkl	       song_id_map.pkl	     with_val_train.json
model.py	       tag_id_map.pkl
most_popular.py        tags.model
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/melon-playlist-continuation


In [ ]:
# myals.py line 23
# movielens.py line 53

In [1]:
%pip install fire
%pip install implicit

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
%cd /content/gdrive/My\ Drive/melon-playlist-continuation
!pwd

/content/gdrive/My Drive/melon-playlist-continuation
/content/gdrive/My Drive/melon-playlist-continuation


In [110]:
run movielens.py --model myals --K 900

INFO:implicit:read data file in 6.9613378047943115
DEBUG:implicit:weighting matrix by bm25_weight
DEBUG:implicit:training model myals
DEBUG:implicit:Calculated transpose in 0.456s
DEBUG:implicit:Initialized factors in 7.480015277862549
DEBUG:implicit:Running 15 ALS iterations


tensor([[0.8795, 0.8200, 0.7786, 0.7757, 0.7624]], device='cuda:0')



DEBUG:implicit:trained model 'myals' in 284.1659598350525
DEBUG:implicit:calculating top movies


In [7]:
import pickle
with open('tag_id_map.pkl', 'rb') as f:
    my_dict = pickle.load(f)
    my_dict_inv = {} 
    for k, v in my_dict.items():
        my_dict_inv[v] = k    
f.close()


In [197]:
import time
import pandas as pd
from arena_util import load_json
from arena_util import write_json
from arena_util import remove_seen
from tqdm import tqdm

train = pd.read_json('./arena_data/orig/train.json', encoding='UTF-8')
val = pd.read_json('./arena_data/questions/val.json', encoding='UTF-8')
song_meta = pd.read_json('./res/song_meta.json', encoding='UTF-8')
most_results = pd.read_json('./arena_data/results/results.json', encoding='UTF-8')
w2v_results = pd.read_json('./omg2.json', encoding='UTF-8')

'''
#train = pd.read_json('./arena_data/orig/train.json', encoding='UTF-8')
train = pd.read_json('./res/train.json', encoding='UTF-8')
#train = pd.read_json('./arena_data/tv.json', encoding='UTF-8')
val = pd.read_json('./res/val.json', encoding='UTF-8')
song_meta = pd.read_json('./res/song_meta.json', encoding='UTF-8')
most_results = pd.read_json('./omg2_real.json', encoding='UTF-8')
w2v_results = pd.read_json('./omg2_real.json', encoding='UTF-8')
'''

song_dict = {} # plylst id - song id
tag_dic = {} # plylst id - tag
data = pd.concat([train, val])
data = data.set_index('id')

song_dict = data['songs'].to_dict()
tag_dict = data['tags'].to_dict()


In [198]:
from tqdm import tqdm
import pickle
with open("./tmp1.pkl", 'rb') as f:
#f = open("./tmp1000.pkl", 'rb')
#f = open("./tmp1000_real.pkl", 'rb')
    x = pickle.load(f)

with open("./tmp2.pkl", 'rb') as f:
#f = open("./tmp2000.pkl", 'rb')
#f = open("./tmp2000_real.pkl", 'rb')
    y = pickle.load(f)

cnt = 0
answers1 = []
for index, q in tqdm(val.iterrows()):
    if q['songs'] != [] or q['tags'] != []:
        answers1.append({
            "id": q["id"],
            "songs": remove_seen(q["songs"], x[cnt])[:200],
            "tags": remove_seen(q["tags"], [my_dict_inv[_] for _ in y[cnt]])[:20],
        })
        cnt += 1
    
    else:
        answers1.append({
          "id": w2v_results.loc[index]["id"],
          "songs": w2v_results.loc[index]['songs'],
          "tags": w2v_results.loc[index]["tags"],
        })

23015it [00:05, 3977.45it/s]


In [95]:
#run movielens.py --model bm25 --K 2

INFO:implicit:read data file in 8.404771327972412
DEBUG:implicit:training model bm25


2


DEBUG:implicit:trained model 'bm25' in 19.449828624725342
DEBUG:implicit:calculating top movies
DEBUG:implicit:calculating similar playlists
  0%|          | 0/153429 [00:00<?, ?it/s]

100%|██████████| 153429/153429 [00:00<00:00, 342628.10it/s]


In [96]:
'''
from tqdm import tqdm
similar_playlist = {}
f = open("./similar-playlist.txt", 'r')
try:
  while True:
    a, b, _ = f.readline().split()
    a = int(a)
    b = int(b)
    if a==b:
      continue
    try:
      similar_playlist[a].append(b)
    except:
      similar_playlist[a] = [b]
except:
  print("end")
f.close()

answers2 = []
for index, q in tqdm(val.iterrows()):
    try:
        most_id = similar_playlist[q['id']]
        get_song = []
        get_tag = []
        for ID in most_id:
            get_song += song_dict[int(ID)]
            get_tag += tag_dict[int(ID)]
        get_song = list(pd.value_counts(get_song)[:200].index)
        get_tag = list(pd.value_counts(get_tag)[:20].index)
        
        answers2.append({
            "id": q["id"],
            "songs": remove_seen(q["songs"], get_song)[:200],
            "tags": remove_seen(q["tags"], get_tag)[:20],
        })
    
    except KeyError:
        answers2.append({
          "id": w2v_results.loc[index]["id"],
          "songs": w2v_results.loc[index]['songs'],
          "tags": w2v_results.loc[index]["tags"],
        }) 
'''

104it [00:00, 516.80it/s]

end


23015it [00:43, 534.60it/s]


In [97]:
#run movielens.py --model bm25 --K 6

INFO:implicit:read data file in 7.703733444213867
DEBUG:implicit:training model bm25


6


DEBUG:implicit:trained model 'bm25' in 19.651859760284424
DEBUG:implicit:calculating top movies
DEBUG:implicit:calculating similar playlists
  0%|          | 0/153429 [00:00<?, ?it/s]

100%|██████████| 153429/153429 [00:00<00:00, 230391.48it/s]


In [98]:
'''
from tqdm import tqdm
similar_playlist = {}
f = open("./similar-playlist.txt", 'r')
try:
  while True:
    a, b, _ = f.readline().split()
    a = int(a)
    b = int(b)
    if a==b:
      continue
    try:
      similar_playlist[a].append(b)
    except:
      similar_playlist[a] = [b]
except:
  print("end")
f.close()

answers3 = []
for index, q in tqdm(val.iterrows()):
    try:
        most_id = similar_playlist[q['id']]
        get_song = []
        get_tag = []
        for ID in most_id:
            get_song += song_dict[int(ID)]
            get_tag += tag_dict[int(ID)]
        get_song = list(pd.value_counts(get_song)[:200].index)
        get_tag = list(pd.value_counts(get_tag)[:20].index)
        
        answers3.append({
            "id": q["id"],
            "songs": remove_seen(q["songs"], get_song)[:200],
            "tags": remove_seen(q["tags"], get_tag)[:20],
        })
    
    except KeyError:
        answers3.append({
          "id": w2v_results.loc[index]["id"],
          "songs": w2v_results.loc[index]['songs'],
          "tags": w2v_results.loc[index]["tags"],
        }) 
'''

52it [00:00, 519.59it/s]

end


23015it [00:44, 513.72it/s]


In [113]:
import pickle
f = open("./bm25K2.pkl", 'rb')
answers2 = pickle.load(f)
f.close()
f = open("./bm25K6.pkl", 'rb')
answers3 = pickle.load(f)
f.close()


In [ ]:
'''answers1 = answers1_
answers2 = answers2_
answers3 = answers3_'''

In [114]:
def date2int(st):
    return int(st[:4]) * 10000 + int(st[5:7]) * 100 + int(st[8:10])
song = load_json("./res/song_meta.json")
for i, a in enumerate(answers1):
    tmp = []
    date = date2int(val.iloc[i]['updt_date'])
    for s in a['songs']:
        d = int(song[s]['issue_date'])
        if d <= date:
            tmp.append(s)
    a['songs'] = tmp
'''
for i, a in enumerate(answers2):
    tmp = []
    date = date2int(val.iloc[i]['updt_date'])
    for s in a['songs']:
        d = int(song[s]['issue_date'])
        if d <= date:
            tmp.append(s)
    a['songs'] = tmp
for i, a in enumerate(answers3):
    tmp = []
    date = date2int(val.iloc[i]['updt_date'])
    for s in a['songs']:
        d = int(song[s]['issue_date'])
        if d <= date:
            tmp.append(s)
    a['songs'] = tmp
'''

In [115]:
# 15 30 50

answers = []
for i in range(len(answers1)):
  tmp = {}
  tmp_song = {}
  tmp_tag = {}
  for n, j in enumerate(answers1[i]['songs']):
      try:
        tmp_song[j] += 1 / (n+15)
      except:
        tmp_song[j] = 1 / (n+15)
  for n, j in enumerate(answers2[i]['songs']):
      try:
        tmp_song[j] += 1 / (n+30)
      except:
        tmp_song[j] = 1/ (n+30)
  for n, j in enumerate(answers3[i]['songs']):
      try:
        tmp_song[j] += 1 / (n+50)
      except:
        tmp_song[j] = 1 / (n+50)
  '''
  for n, j in enumerate(answers4[i]['songs']):
      try:
        tmp_song[j] += 1 / (n+200)
      except:
        tmp_song[j] = 1 / (n+30) / 2
  '''
  
          
  sorted_keys = sorted(tmp_song.items(), reverse=True, key=lambda item: item[1])
  sorted_keys = [k for (k, v) in sorted_keys]

  tmp['id'] = answers1[i]['id']
  tmp['songs'] = sorted_keys[:100]


  for n, j in enumerate(answers1[i]['tags']):
      try:
        tmp_tag[j] += 1 / (n+15)
      except:
        tmp_tag[j] = 1 / (n+15)
  for n, j in enumerate(answers2[i]['tags']):
      try:
        tmp_tag[j] += 1 / (n+30)
      except:
        tmp_tag[j] = 1 / (n+30)
  for n, j in enumerate(answers3[i]['tags']):
      try:
        tmp_tag[j] += 1 / (n+50)
      except:
        tmp_tag[j] = 1/ (n+50)

  sorted_keys = sorted(tmp_tag.items(), reverse=True, key=lambda item: item[1])
  sorted_keys = [k for (k, v) in sorted_keys]




  tmp['tags'] = sorted_keys[:10]
  answers.append(tmp)

for i, q in val.iterrows():
    answers[i] = {
        "id": q["id"],
        "songs": remove_seen(q["songs"], answers[i]['songs'])[:100],
        "tags": remove_seen(q["tags"], answers[i]['tags'])[:10],
    }

cnt = 0
for n, q in enumerate(answers):
    if len(q['songs'])>100:
        q['songs'] = q['songs'][:100]
    if len(q['tags'])>10:
        q['tags'] = q['tags'][:10]
    if len(q['songs'])!=100:
        cnt += 1
        answers[n]['songs'] += remove_seen(q['songs'], most_results.loc[n]['songs'])[:100-len(q['songs'])]
    if len(q['tags'])!=10:
        answers[n]['tags'] += remove_seen(q['tags'], most_results.loc[n]['tags'])[:10-len(q['tags'])]
for i, q in val.iterrows():
    if(len(q['songs'])==0):
        answers[i]['songs'] = w2v_results.loc[i]['songs'][:100]


In [116]:
write_json(answers, "./results/myans.json")

In [118]:
run evaluate.py evaluate --gt_fname=arena_data/answers/val.json --rec_fname=arena_data/results/myans.json
#0.2928

Music nDCG: 0.292374
Tag nDCG: 0.410661
Score: 0.310117


In [140]:
import arena_util as au
v= au.load_json('./arena_data/questions/val.json')
st = {}
f = open("./ndcg.txt", 'r')
for i in range(len(v)):
    nd = float(f.readline())
    try:
        st[(len(v[i]['songs']), len(v[i]['tags']))].append(nd)
    except:
        st[(len(v[i]['songs']), len(v[i]['tags']))] = [nd]

for i in range(101):
  print("노래%3d개"%(i), end = " " )
  for j in range(11):
    try:
      #print("%.2lf"%(np.mean(st[(i, j)])), end=" ")
      print(len(st[(i,j)]), end=" ")
    except:
      print(" x  ", end=" ")
  print()


노래  0개 811 535 250 145 131 171  x    x    x    x    x   
노래  1개 7 2  x    x    x    x    x    x    x    x    x   
노래  2개 37 10 5  x    x    x    x    x    x    x    x   
노래  3개 51 14 5 3  x    x    x    x    x    x    x   
노래  4개 87 34 13 6 2 1  x    x    x    x    x   
노래  5개 207 96 36 13 8 1  x    x    x    x    x   
노래  6개 193 78 26 16 4 8  x    x    x    x    x   
노래  7개 229 82 36 11 13 11  x    x    x    x    x   
노래  8개 192 74 35 24 20 12  x    x    x    x    x   
노래  9개 327 99 64 32 32 24  x    x    x    x    x   
노래 10개 376 150 66 61 67 46  x    x    x    x    x   
노래 11개 160 63 40 25 23 15  x    x    x    x    x   
노래 12개 151 54 33 19 22 23  x    x    x    x    x   
노래 13개 153 78 33 19 20 16  x    x    x    x    x   
노래 14개 165 59 36 19 30 26  x    x    x    x    x   
노래 15개 208 62 41 27 41 36  x    x    x    x    x   
노래 16개 97 50 29 16 18 15  x    x    x    x    x   
노래 17개 103 39 18 15 14 15  x    x    x    x    x   
노래 18개 92 40 17 10 10 18  x    x    x    x    x   
노래 19개

In [ ]:
answers1 : myALS
answers2 : 
answers3 : bm25-K=2
answers4 : bm25-K=6

In [109]:
import myals
import imp
imp.reload(myals)


<module 'myals' from '/content/gdrive/My Drive/melon-playlist-continuation/myals.py'>

In [144]:
%pip install sentencepiece

     |████████████████████████████████| 1.1MB 4.2MB/s 


In [145]:
import sentencepiece as spm

In [146]:
import arena_util as au

train = au.load_json('./res/train.json')
val = au.load_json('./res/val.json')
test = au.load_json('./res/test.json')


In [156]:
f = open('tagstext.txt', 'w', encoding='utf8')
for i in train:
  if(i['plylst_title']!=''):
    f.write(i['plylst_title']+"\n")
for i in val:
  if(i['plylst_title']!=''):
    f.write(i['plylst_title']+"\n")
for i in test:
  if(i['plylst_title']!=''):
    f.write(i['plylst_title']+"\n")
f.close()

In [158]:
spm.SentencePieceTrainer.Train('--input=tagstext.txt --model_prefix=tags --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [161]:
import csv
vocab_list = pd.read_csv('tags.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
1431,▁유명,-1428
3530,렉,-3527
4769,˃,-4766
4034,항,-4031
2532,았던,-2529
2921,▁느낌적인,-2918
973,▁뜨,-970
807,▁자장가,-804
4105,덥,-4102
4073,빨,-4070


In [162]:
sp = spm.SentencePieceProcessor()
vocab_file = "tags.model"
sp.load(vocab_file)

True

In [185]:
line = train[8]['plylst_title']
print(line)
print(sp.encode_as_pieces(line))
print(sp.encode_as_ids(line))
print()

DANCING IN THE MOON-LIGHT .01
['▁D', 'AN', 'C', 'ING', '▁IN', '▁THE', '▁M', 'O', 'ON', '-', 'L', 'I', 'G', 'H', 'T', '▁.01']
[207, 1313, 3399, 3248, 3212, 1372, 110, 3348, 881, 3395, 3463, 3449, 3486, 3443, 3365, 1789]



In [166]:
def process_to_IDs_in_sparse_format(sp, sentences):
  # An utility method that processes sentences with the sentence piece processor
  # 'sp' and returns the results in tf.SparseTensor-similar format:
  # (values, indices, dense_shape)
  ids = [sp.EncodeAsIds(x) for x in sentences]
  max_len = max(len(x) for x in ids)
  dense_shape=(len(ids), max_len)
  values=[item for sublist in ids for item in sublist]
  indices=[[row,col] for row in range(len(ids)) for col in range(len(ids[row]))]
  return (values, indices, dense_shape)